In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

## やりたいこと
+ サブクエリのネストにならずに複数のjoin条件をSQL実装する

In [3]:
%%sql
drop table if exists Foo, Bar, Accounts;
CREATE TABLE Accounts
 (acct_nbr INTEGER NOT NULL PRIMARY KEY);

CREATE TABLE Foo
 (acct_nbr INTEGER NOT NULL
    REFERENCES Accounts(acct_nbr),
  foo_qty INTEGER NOT NULL);

CREATE TABLE Bar
(acct_nbr INTEGER NOT NULL
    REFERENCES Accounts(acct_nbr),
 bar_qty INTEGER NOT NULL);

INSERT INTO Accounts VALUES (1);
INSERT INTO Accounts VALUES (2);
INSERT INTO Accounts VALUES (3);
INSERT INTO Accounts VALUES (4);

INSERT INTO Foo VALUES (1, 10);
INSERT INTO Foo VALUES (2, 20);
INSERT INTO Foo VALUES (2, 40);
INSERT INTO Foo VALUES (3, 80);

INSERT INTO Bar VALUES (2, 160);
INSERT INTO Bar VALUES (3, 320);
INSERT INTO Bar VALUES (3, 640);
INSERT INTO Bar VALUES (3, 1);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%%sql
-- こちらは普通にやった時の回答
select tmp.acct_nbr, tot_foo_qty, sum(bar_qty) as tot_bar_qty
from (
    select a.acct_nbr, sum(foo_qty)
    from Accounts as a
    left join Foo as f
    on a.acct_nbr = f.acct_nbr
    group by a.acct_nbr
) as tmp(acct_nbr, tot_foo_qty)
left join Bar as b
on tmp.acct_nbr = b.acct_nbr
group by tmp.acct_nbr, tot_foo_qty
order by acct_nbr

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


acct_nbr,tot_foo_qty,tot_bar_qty
1,10,None
2,60,160
3,80,961
4,None,None


In [24]:
%%sql
select acct_nbr, sum(foo_qty), sum(bar_qty)
from  (
    (
        select a.acct_nbr, foo_qty, bar_qty
        from Accounts as a
        left join (select acct_nbr, foo_qty, 0 as bar_qty from foo) as f
        on a.acct_nbr = f.acct_nbr
    )
    union all
    (
        select a.acct_nbr, foo_qty, bar_qty
        from Accounts as a
        left join (select acct_nbr, 0 as foo_qty, bar_qty from bar) as b
        on a.acct_nbr = b.acct_nbr
    )
) as tmp
group by acct_nbr
order by acct_nbr

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


acct_nbr,sum,sum_1
1,10,0
2,60,160
3,80,961
4,None,None
